In [ ]:
from ntscraper import Nitter
from helper_functions import compute_unique_primary_key
from datetime import datetime, timedelta
from db import Model, Session, engine
import models
from sqlalchemy.orm import clear_mappers
from sqlalchemy.exc import IntegrityError
from psycopg2.errors import UniqueViolation
import pytz

In [ ]:
scraper = Nitter(log_level=1)

In [ ]:
instance ="https://nitter.woodland.cafe"
#since = "2023-01-01"
since = "2021-11-05"
until = "2021-11-06"
GTB_tweets = scraper.get_tweets("@gtbank",number=5, 
                                mode='term',instance=instance,
                                since=since,
                               until=until)

In [ ]:
len(GTB_tweets['tweets'])

In [ ]:
GTB_tweets['tweets'][0].keys()

In [ ]:
tweet = GTB_tweets['tweets'][0]

In [ ]:
tweet

In [ ]:
store_hash = []
for tweet in GTB_tweets['tweets']:
    input_str = tweet.get('date',None)+tweet.get('user',{}).get('profile_id',None)+str(len(tweet.get('text',None)))
    store_hash.append(compute_unique_primary_key(input_str))

In [ ]:
len(store_hash[0])

In [ ]:
compute_unique_primary_key(input_str)

In [ ]:
len(dir(models.Tweet))

In [ ]:
Model.metadata.drop_all(engine)
Model.metadata.create_all(engine)

In [ ]:
company_data = {'id':1,
                'name':'Guaranty Trust Bank',
                'nickname': 'gtbank',
                'country': 'Nigeria',
                'industry': 'Banking',
                'username': '@gtbank'
}


In [ ]:
company = models.Company(**company_data)

In [ ]:
company

In [ ]:
with Session() as session:
    with session.begin():
        session.add(company)

In [ ]:
def get_tweets(since,until,username):
    tweets = scraper.get_tweets(username,number=1000, 
                                mode='term',instance=instance,
                                since=since,
                               until=until)
    return tweets

In [ ]:
def process_tweets(tweets,company_id):
    store_tweets = []
    for tweet in tweets['tweets']:
        input_str = tweet.get('date',None)+tweet.get('user',{}).get('profile_id',None)+str(len(tweet.get('text',None)))
        store_hash = compute_unique_primary_key(input_str)
        date_str = tweet.get('date',None)
        if date_str:
            # Parse the datetime string, omitting the 'UTC' part
            parsed_date = datetime.strptime(date_str, '%b %d, %Y · %I:%M %p UTC')
            # Define the UTC timezone
            utc = pytz.timezone('UTC')
            date_str = utc.localize(parsed_date)
            
        data = { 'id':str(store_hash),
                'text':tweet.get('text',None),
                'profile_id':tweet.get('user',{}).get('profile_id',None),
                'stats_likes':int(tweet.get('stats',{}).get('likes',0)),
                'stats_retweets':int(tweet.get('stats',{}).get('retweets',0)),
                'company_id': int(company_id),
                'date': date_str
        }
        store_tweets.append(data)
    return store_tweets

In [ ]:
def store_in_db(tweets):
    with Session() as session:
        for tweet in tweets:
            tweet_item = models.Tweet(**tweet)
            try:
                session.add(tweet_item)
                session.commit()  # Commit each tweet individually
            except IntegrityError:
                session.rollback()  # Rollback if there's an error
                print(f"Tweet record already exists")  # More informative error message


In [ ]:
def weekly_intervals_caller(since, until,username,company_id):
    start_date = datetime.strptime(since, "%Y-%m-%d")
    end_date = datetime.strptime(until, "%Y-%m-%d")
    temp_until = start_date

    while temp_until < end_date:
        print(start_date)
        temp_until = start_date + timedelta(days=7)
        # Adjust temp_until to not exceed the end_date
        if temp_until > end_date:
            temp_until = end_date

        tweets = get_tweets(since=start_date.strftime("%Y-%m-%d"), 
                            until=temp_until.strftime("%Y-%m-%d"),
                           username=username)
        processed_tweets = process_tweets(tweets,company_id)
        store_in_db(processed_tweets)
        
        # Set the new start_date for the next iteration
        start_date = temp_until

In [ ]:
instance ="https://nitter.woodland.cafe"
since = "2023-01-01"
until = "2023-01-30"

In [ ]:
weekly_intervals_caller(since='2021-11-04',until='2023-11-20',username='@gtbank',company_id=1)